In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAPI_KEY") # (Optional), if OpenAI Model is used

MODEL = "mistral:latest" # Name of the model used by Ollama

COLLECTION_NAME = 'scouting' # Name of the Collection to be created

DIMENSION = 1536 # Dimension of the embeddings

URI = 'http://localhost:19530' # Connection parameters for the Milvus Server

In [27]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAIEmbeddings

model = Ollama(model=MODEL)
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [28]:
# Configure the prompt template that is used to ask the LLM

from langchain_core.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
print(prompt.format(context="Here is some context", question="Here is a question"))


Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
Here is some context
</context>

<question>
Here is a question
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:


In [29]:
# Use Milvus as Vectorstore

from langchain_community.vectorstores import Milvus

connection_args = {'uri': URI }

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    vector_field="embeddings",
    primary_field="id",
    auto_id=True
    
)


In [30]:
# Convert the vector store to a retriever
retriever = vectorstore.as_retriever()
# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
query = "Ich brauche einen Rechtsverteidiger, der konstant über das ganze Spiel durchspielt. Er sollte auch im Spiel nach vorne gut sein"
res = rag_chain.invoke(query)
res

' In your search for a right-back who performs consistently throughout the game, consider players around 25 years old with potential for improvement, such as those mentioned in the context. They demonstrate strong defensive skills, allowing few opportunities for opponents and contribute offensively through precise passes and well-timed advances. However, they may struggle against fast wingers due to a perceived weakness in their end speed. To improve the precision of long balls, you might also consider players with this specific skill set to be trained further.'

In [32]:
from langchain_core.documents.base import Document

def extract_metadata(doc: Document) -> dict:
    return doc.metadata

In [33]:
# How to work with meta data from our query
retrived_documents = retriever.invoke(query)
retrived_documents

[Document(page_content='\n Wieder als rechter Verteidiger eingesetzt, zeigte er eine beständige Leistung über die gesamte Spielzeit.\nDefensiv sehr solide, ließ kaum Flanken und gegnerische Durchbrüche zu.\nOffensiv setzte er Akzente durch präzise Flanken und gute Vorstöße.\nSeine Ausdauer und Einsatzbereitschaft waren erneut beeindruckend.\nSeine Endgeschwindigkeit bleibt eine Schwäche, die gegen schnelle Gegenspieler problematisch sein könnte.\nMit 25 Jahren hat er noch Verbesserungspotenzial.\n ', metadata={'id': 450236041423488387, 'scout_id': '0987', 'player_id': 'bf1234567890abcdef123456', 'player_transfermarkt_id': '951357', 'grade_rating': 0.699999988079071, 'grade_potential': 0.75}),
 Document(page_content='\n Spielte als rechter Verteidiger und zeigte eine konstante Leistung über 90 Minuten.\nWar sicher in der Defensive und ließ kaum Flanken zu.\nOffensiv setzte er Akzente mit präzisen Pässen und gut getimten Vorstößen.\nSeine Ausdauer und Einsatzbereitschaft waren bemerkensw

In [34]:
metadata = extract_metadata(retrived_documents[0])
metadata

{'id': 450236041423488387,
 'scout_id': '0987',
 'player_id': 'bf1234567890abcdef123456',
 'player_transfermarkt_id': '951357',
 'grade_rating': 0.699999988079071,
 'grade_potential': 0.75}

In [35]:
# Transfermarkt Link
print(f"Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/{metadata['player_transfermarkt_id']}")

Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/951357


In [ ]:
print